In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy

In [3]:
# Dep_credit: nonrefundable portion
# c07220: refundable portion, not yet calculated as ACTC
# c11070: portion actually refunded
# TODO: Create a function that takes a subpopulation and spits out total change in revenue, money going to CTC, eitc, per person, etc.

def taxdf(calc, namelist, varlist):
    calcu=copy.deepcopy(calc)
    tdf=pd.DataFrame()
    for v in range (0, len(namelist)):
        tdf[namelist[v]]=getattr(calcu.records,varlist[v])
    return tdf

In [21]:
def percentile(df, percentile, var):
    df = df.sort_values(var, axis = 0)
    df['cumsum'] = np.cumsum(df['s006'])
    return df[df['cumsum'] <= (percentile/100) * df['cumsum'].max()].iloc[-1][var]

In [5]:
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\TCJA_Reconciliation.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
reform = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})
reform_noctc = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})

for y in range(2017,2027):
    if y in reform_noctc.keys():
        reform_noctc[y]['_CTC_c'] = [1000]
        reform_noctc[y]['_DependentCredit_Child_c'] = [0]
        reform_noctc[y]['_ACTC_Income_thd'] = [3000]
        reform_noctc[y]['_CTC_ps'] = [[75000, 110000, 55000, 75000, 75000]]

# Create calculator for TCJA Reconciliation plan, minus refundable child tax credit

pol_noctc = tc.Policy()
pol_noctc.implement_reform(reform_noctc)

calc = tc.Calculator(policy = pol_noctc, records = tc.Records())
calc.advance_to_year(2018)
calc.calc_all()

# Create calculator for TCJA Reconciliation plan EXPANSION

pol = tc.Policy()
pol.implement_reform(reform)

calc1 = tc.Calculator(records = tc.Records(), policy = pol)
calc1.advance_to_year(2018)
calc1.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [6]:
nlist = ['c00100', 'standard', 'c04470', 'iitax','combined','payrolltax','s006','MARS', 'n24', 'XTOT', 'c11070', 'c09600', 'refund']
vlist = ['c00100', 'standard', 'c04470', 'iitax','combined','payrolltax','s006','MARS', 'n24', 'XTOT', 'c11070', 'c09600', 'refund']
calcdf = taxdf(calc=calc, namelist = nlist, varlist = nlist)
calcdf1 = taxdf(calc=calc1, namelist = nlist, varlist = nlist)

In [7]:
# GET TOTAL MONEY SPENT ON CHILD TAX REFUNDS
# MAKE SINGLE PARENT DUMMY ARRAY
def costdiff(no_expansion, expansion, subpop):
    expansion['w_iitaxdiff'] = (expansion['iitax'] - no_expansion['iitax'])*no_expansion['s006']
    expansion['w_c11070'] = expansion['c11070'] * no_expansion['s006']
    expansion_sp = copy.deepcopy(expansion[subpop])
    singleparent = ((expansion_sp['MARS'] == 1) | (expansion_sp['MARS'] == 4)) & (expansion_sp['n24']  >  0)
    num_single_parents = expansion_sp[singleparent]['s006'].sum()
    total = expansion_sp['s006'].sum()
    print("Number of Beneficiaries in Group that are Single or Head of Household with Kids: " +
          '{:,}'.format(round(num_single_parents)))
    print("Number of Beneficiaries in Group: " +
          '{:,}'.format(round(total)))
    print("Amount of Revenue Going towards Taxpayers in Group: " + 
          '{:,}'.format(round(expansion_sp['w_iitaxdiff'].sum())))
    print('Amount of Revenue Going Towards Single Parent Beneficiaries: ' +
          '{:,}'.format(round(expansion_sp[singleparent]['w_iitaxdiff'].sum())))
    print('Total ACTC Benefits Claimed by Group: ' +
          '{:,}'.format(round(expansion[subpop]['w_c11070'].sum())))
    print('Total ACTC Benefits Claimed by Single Parents in Group: ' +
          '{:,}'.format(round(expansion_sp[singleparent]['w_c11070'].sum())))    

In [39]:
# CHECK IF REFUND RECEIVING BENEFICIARIES ARE FOR CURRENT LAW, TCJA, or TCJA without CTC expansion
pop1 = [(calcdf['iitax'] < 0), 'Refund-Receiving Beneficiaries']
pop2 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(5000, 15000))), 'Refund-Receiving Beneficiaries, AGI of 5-15 Thousand']
pop3 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(15000, 25000))), 'Refund-Receiving Beneficiaries, AGI of 15-25 Thousand']
pop4 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(19000, 21000))), 'Refund-Receiving Beneficiaries, AGI of 19-21 Thousand']

for x in [pop1, pop2, pop3, pop4]:
    print(x[1])
    costdiff(calcdf, calcdf1, x[0])
    print('\n')

Refund-Receiving Beneficiaries
Number of Beneficiaries in Group that are Single or Head of Household with Kids: 14,210,587.0
Number of Beneficiaries in Group: 32,336,414.0
Amount of Revenue Going towards Taxpayers in Group: -15,345,656,793.0
Amount of Revenue Going Towards Single Parent Beneficiaries: -10,008,230,773.0
Total ACTC Benefits Claimed by Group: 34,216,923,255.0
Total ACTC Benefits Claimed by Single Parents in Group: 22,755,933,414.0


Refund-Receiving Beneficiaries, AGI of 5-15 Thousand
Number of Beneficiaries in Group that are Single or Head of Household with Kids: 3,702,256.0
Number of Beneficiaries in Group: 10,295,002.0
Amount of Revenue Going towards Taxpayers in Group: -809,779,893.0
Amount of Revenue Going Towards Single Parent Beneficiaries: -671,929,033.0
Total ACTC Benefits Claimed by Group: 4,956,762,216.0
Total ACTC Benefits Claimed by Single Parents in Group: 4,081,693,009.0


Refund-Receiving Beneficiaries, AGI of 15-25 Thousand
Number of Beneficiaries in Grou

In [25]:
percentile(calcdf[pop1], percentile=50, var = 'c00100')

16274.535721311191

In [37]:
2091169138.0/1068172.0

1957.7082511056271